- Fine-tune multi-lingual - with both Hindi and Nepali
- Fine tune with Hindi run inference on Nepali 
- Fine tune with Nepali run inference on Hindi
- Fine tune with Nepali run inference on Nepali
- Fine tune with Hindi run inference on Hindi

In [2]:
from kaggle_secrets import UserSecretsClient

In [3]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [4]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 104 (delta 50), reused 62 (delta 15), pack-reused 0
Receiving objects: 100% (104/104), 27.40 MiB | 18.39 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [5]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [6]:
import os
import wandb

os.environ["WANDB_PROJECT"]="llama-finetune-nepali"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
wandb_token = user_secrets.get_secret("WANDB_KEY")
wandb.login(key= wandb_token)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
from datasets import load_dataset, load_metric
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-16 20:29:23.689276: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-16 20:29:23.689421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-16 20:29:23.947179: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
split_ratio = 0.2
seed = 42

In [21]:
dataset_name = "CohereForAI/aya_dataset"
aya_dataset = load_dataset(dataset_name)
dataset_type = "aya"

In [22]:
aya_nepali = aya_dataset.filter(lambda example: example['language'] == "Nepali")
aya_nepali

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
        num_rows: 4002
    })
    test: Dataset({
        features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
        num_rows: 0
    })
})

In [23]:
aya_hindi = aya_dataset.filter(lambda example: example['language'] == "Hindi")
aya_hindi

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
        num_rows: 1153
    })
    test: Dataset({
        features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
        num_rows: 0
    })
})

In [24]:
lang_multi = "Nepali"
aya_nepali_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4002 [00:00<?, ? examples/s]

In [25]:
lang_multi = "Hindi"
aya_hindi_train, _ = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1153 [00:00<?, ? examples/s]

In [26]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
alpaca_nepali = load_dataset(dataset_name)

In [27]:
alpaca_nepali_filtered = alpaca_nepali.filter(lambda example: example['output'] is not None and example['output'] != "")
alpaca_nepali_filtered = alpaca_nepali_filtered.filter(lambda example: example['instruction'] != "" and example['instruction'] is not None)
alpaca_nepali_filtered = alpaca_nepali_filtered.filter(lambda example: example['input'] is not None)
alpaca_nepali_train, _ = sample_data(alpaca_nepali_filtered, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

TypeError: expected str, bytes or os.PathLike object, not DatasetDict

In [15]:
dataset_name = load_dataset("iamshnoo/alpaca-cleaned-hindi")

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 51760
    })
})

In [16]:
alpaca_hindi_filtered = alpaca_hindi.filter(lambda example: example['output'] is not None and example['output'] != "")
alpaca_hindi_filtered = alpaca_hindi_filtered.filter(lambda example: example['instruction'] != "" and example['instruction'] is not None)
alpaca_hindi_filtered = alpaca_hindi_filtered.filter(lambda example: example['input'] is not None)
alpaca_hindi_train, _ = sample_data(alpaca_hindi_filtered, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 51760
    })
})

In [ ]:
#Neapli
prompt = """Below is an instruction in Hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in Hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
#LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN


def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


from datasets import load_dataset
dataset = load_dataset("iamshnoo/alpaca-cleaned-bengali", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 1000,                #### Setting max_steps for 1000. (1-1000)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        save_steps=500,                 ### Checkpoint will be save after every 500 steps
        optim = "adamw_8bit",
        weight_decay = 0.01,
        report_to="wandb", 
        run_name="llama3_alpaca_nep", 
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs_alpaca_nep",   # Saving the checkpoints to outputs folder
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
wandb.finish()